# Lab 11 - Pruning Parcels

## Tasks

#### Task 1 - Proptype on 2004 parcel data

Build a query that will filter the 2004 parcel file

1. Read in the 2004 parcel file
2. Create a column that contains True if the parcel is close to our 49 lakes (see lab 10) and flase otherwise. (Hint: create `lat_long` -> map to `code` -> check `isin` for the lakes we identified in lab 10).
3. Create a query that selects the correct columns, filters on the new column, then adds lake_names, lake_codes, and lake distance.  Finally filter by distance to the lake.

How much have we reduced the size of the parcel file?  Can you safely union all the files after these filters?

#### Task 2 - Read all 11 files and apply the process defined in the last task


#### Task 3 - Union all 11 files then write out the result.

In [1]:
import pandas as pd
from dfply import *

In [2]:
dtypes = {'centroid_lat': str, 'centroid_long': str}
parcel2004 = pd.read_csv('./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt', sep = '|', dtype = common_parcel_types)

NameError: name 'common_parcel_types' is not defined

In [ ]:
from project_data_Buske import lake_names_to_keep

In [3]:
from project_data_Buske import lake_codes_to_keep

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (2,3,4,6,7,11,15,18,24,28,29,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
from project_data_Buske import latlong_to_code, latlong_to_distance, latlong_to_name, common_columns

In [5]:
cols_to_keep = ['ACRES_DEED',
                'ACRES_POLY',
                'AGPRE_ENRD',
                'AG_PRESERV',
                'BASEMENT',
                'CITY',
                'COOLING',
                'DWELL_TYPE',
                'EMV_BLDG',
                'EMV_LAND',
                'FIN_SQ_FT',
                'GARAGE', 
                'GARAGESQFT', 
                'GREEN_ACRE',
                'HOMESTEAD',
                #'ID',
                'LANDMARK',
                'OWN_ADD_L1',
                'OWN_ADD_L2',
                'OWN_ADD_L3',
                'PARC_CODE',
                'PIN', 
                'SALE_VALUE', 
                'SPEC_ASSES',
                'TAX_CAPAC', 
                'TAX_EXEMPT', 
                'TOTAL_TAX', 
                'USE1_DESC',
                'USE2_DESC',
                'USE3_DESC',
                'USE4_DESC',
                'WSHD_DIST',
                'XUSE1_DESC',
                'XUSE2_DESC',
                'XUSE3_DESC',
                'XUSE4_DESC',
                'YEAR_BUILT',
                'Year',
                'centroid_lat',
                'centroid_long']

In [6]:
def col_zip(*cols):
    return pd.Series(zip(*cols))

In [7]:
rows_to_keep = (col_zip(parcel2004.centroid_lat,
                        parcel2004.centroid_long)
                 .map(latlong_to_code)
                 .isin(lake_codes_to_keep)
                )

NameError: name 'parcel2004' is not defined

In [ ]:
rows_to_keep.value_counts()

In [ ]:
from functoolz import pipeable
process_parcel_file = pipeable(lambda df:
                               (df
                                   >> select(cols_to_keep)
                                   >> mutate(lat_long = pd.Series(zip(df.centroid_lat,
                                                                      df.centroid_long)))
                                   >> mutate(rows_to_keep = (X.lat_long
                                                                .map(latlong_to_code)
                                                                .isin(lake_codes_to_keep)))
                                   >> filter_by(X.rows_to_keep) 
                                   >> mutate(lake_code = X.lat_long.map(latlong_to_code),
                                             lake_name = X.lat_long.map(latlong_to_name),
                                             lake_dist = X.lat_long.map(latlong_to_distance))
                                   >> filter_by(X.lake_dist <= 1600)
                                   
                               
                               ))

In [ ]:
process_parcel_file(parcel2004) >> head

In [129]:
read_parcel_file = pipeable(lambda f: pd.read_csv(f, sep = '|', dtype = dtypes))

In [130]:
from glob import glob
files = glob('./data/MinneMUDAC_raw_files/20**_metro_tax_parcels.txt')[2:-1]
files

['./data/MinneMUDAC_raw_files/2004_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2005_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2006_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2007_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2008_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2010_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2011_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2012_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2013_metro_tax_parcels.txt',
 './data/MinneMUDAC_raw_files/2014_metro_tax_parcels.txt']

In [131]:
process_files = [f >> read_parcel_file >> process_parcel_file for f in files]
process_files[0]

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,20,22,24,25,26,27,28,29,30,31,32,33,36,37,38,44,45,48,49,50,52,53,55,56,57,58,59,60,61,62,63,64,67) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,19,20,22,23,24,25,26,27,28,29,30,31,32,35,36,37,38,43,44,47,48,49,51,52,54,55,56,57,58,59,60,61,62,63,66) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,19,20,22,23,24,25,26,27,28,29,30,31,32,35,36,37,38,43,44,47,48,49,51,52,54,55,56,57,58,59,60,61,62,63) have mix

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AG_PRESERV,BASEMENT,CITY,COOLING,DWELL_TYPE,EMV_BLDG,EMV_LAND,...,XUSE4_DESC,YEAR_BUILT,Year,centroid_lat,centroid_long,lat_long,rows_to_keep,lake_code,lake_name,lake_dist
128085,0.00,0.25,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,122500.0,34300.0,...,NaN,1986.0,2004,44.88692,-93.62034,"(44.88692, -93.62034)",True,10001100-01,St. Joe Lake,396.400457
128232,0.00,0.33,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,156000.0,35100.0,...,NaN,1983.0,2004,44.88657,-93.621,"(44.88657, -93.621)",True,10001100-01,St. Joe Lake,396.400457
128266,0.00,0.46,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,145100.0,35100.0,...,NaN,1983.0,2004,44.88636,-93.62101,"(44.88636, -93.62101)",True,10001100-01,St. Joe Lake,396.400457
128288,0.00,0.37,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,167100.0,61200.0,...,NaN,1975.0,2004,44.88632,-93.62051,"(44.88632, -93.62051)",True,10001100-01,St. Joe Lake,396.400457
128357,0.00,0.28,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,158900.0,61200.0,...,NaN,1983.0,2004,44.88609,-93.62151,"(44.88609, -93.62151)",True,10001100-01,St. Joe Lake,396.400457
128373,0.00,0.31,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,118600.0,61200.0,...,NaN,1983.0,2004,44.88603,-93.6212,"(44.88603, -93.6212)",True,10001100-01,St. Joe Lake,396.400457
128381,0.00,0.30,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,169600.0,61200.0,...,NaN,1983.0,2004,44.886,-93.62025,"(44.886, -93.62025)",True,10001100-01,St. Joe Lake,396.400457
128384,0.00,0.32,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,161800.0,61200.0,...,NaN,1984.0,2004,44.88598,-93.62088,"(44.88598, -93.62088)",True,10001100-01,St. Joe Lake,396.400457
128402,0.00,0.31,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,153900.0,61200.0,...,NaN,1983.0,2004,44.88594,-93.62057,"(44.88594, -93.62057)",True,10001100-01,St. Joe Lake,396.400457
128549,0.00,0.22,NaN,N,NaN,CHANHASSEN CITY,NaN,NaN,132000.0,44000.0,...,NaN,1986.0,2004,44.88554,-93.6222,"(44.88554, -93.6222)",True,10001100-01,St. Joe Lake,396.400457


In [139]:
#df = pd.DataFrame(columns=cols_to_keep)
#for f in process_files:
#    df = df >> union_all(f)

In [140]:
#df.shape

In [55]:
@dfpipe
def union_all(left_df, right_df, ignore_index=True):
    return pd.concat([left_df, right_df], ignore_index=ignore_index)

In [135]:
df = pd.DataFrame(columns=cols_to_keep)

In [137]:
[df >> union_all(f) for f in process_files]

/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
/Users/tr8934qw/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning

[        ACRES_DEED  ACRES_POLY AGPRE_ENRD AG_PRESERV BASEMENT  \
 0             0.00        0.25        NaN          N      NaN   
 1             0.00        0.33        NaN          N      NaN   
 2             0.00        0.46        NaN          N      NaN   
 3             0.00        0.37        NaN          N      NaN   
 4             0.00        0.28        NaN          N      NaN   
 5             0.00        0.31        NaN          N      NaN   
 6             0.00        0.30        NaN          N      NaN   
 7             0.00        0.32        NaN          N      NaN   
 8             0.00        0.31        NaN          N      NaN   
 9             0.00        0.22        NaN          N      NaN   
 10            0.00        0.30        NaN          N      NaN   
 11            0.00        0.30        NaN          N      NaN   
 12            0.00        0.32        NaN          N      NaN   
 13            0.00        0.30        NaN          N      NaN   
 14       

In [138]:
df.shape

(868468, 44)

In [ ]:
df.to_csv('./data/MinneMUDAC_raw_files/final_parcels.csv', index = False)

In [85]:
#from sqlalchemy import str, Float, Integer, DateTime
common_parcel_types = {'ACRES_DEED':str,
                       'ACRES_POLY':str,
                       'AGPRE_ENRD':str,
                       'AGPRE_EXPD':str,
                       'AG_PRESERV':str,
                       'BASEMENT':str,
                       'BLDG_NUM':str,
                       'BLOCK':str,
                       'CITY':str,
                       'CITY_USPS':str,
                       'COOLING':str,
                       'COUNTY_ID':str,
                       'DWELL_TYPE':str,
                       'EMV_BLDG':str,
                       'EMV_LAND':str,
                       'EMV_TOTAL':str,
                       'FIN_SQ_FT':str,
                       'GARAGE':str,
                       'GARAGESQFT':str,
                       'GREEN_ACRE':str,
                       'HEATING':str,
                       'HOMESTEAD':str,
                       'HOME_STYLE':str,
                       'ID':str,
                       'LANDMARK':str,
                       'LOT':str,
                       'MULTI_USES':str,
                       'NUM_UNITS':str,
                       'OPEN_SPACE':str,
                       'OWNER_MORE':str,
                       'OWNER_NAME':str,
                       'OWN_ADD_L1':str,
                       'OWN_ADD_L2':str,
                       'OWN_ADD_L3':str,
                       'PARC_CODE':str,
                       'PIN':str,
                       'PLAT_NAME':str,
                       'PREFIXTYPE':str,
                       'PREFIX_DIR':str,
                       'SALE_DATE':str,
                       'SALE_VALUE':str,
                       'SCHOOL_DST':str,
                       'SPEC_ASSES':str,
                       'STREETNAME':str,
                       'STREETTYPE':str,
                       'SUFFIX_DIR':str,
                       'Shape_Area':str,
                       'Shape_Leng':str,
                       'TAX_ADD_L1':str,
                       'TAX_ADD_L2':str,
                       'TAX_ADD_L3':str,
                       'TAX_CAPAC':str,
                       'TAX_EXEMPT':str,
                       'TAX_NAME':str,
                       'TOTAL_TAX':str,
                       'UNIT_INFO':str,
                       'USE1_DESC':str,
                       'USE2_DESC':str,
                       'USE3_DESC':str,
                       'USE4_DESC':str,
                       'WSHD_DIST':str,
                       'XUSE1_DESC':str,
                       'XUSE2_DESC':str,
                       'XUSE3_DESC':str,
                       'XUSE4_DESC':str,
                       'YEAR_BUILT':str,
                       'Year':str,
                       'ZIP':str,
                       'ZIP4':str,
                       'centroid_lat':str,
                       'centroid_long':str}